In [81]:
import requests, json, time
import pandas as pd
%env GOOGLE_APPLICATION_CREDENTIALS=/Users/mariusconjeaud/openstudybuildertests-9497bfb181cb.json


env: GOOGLE_APPLICATION_CREDENTIALS=/Users/mariusconjeaud/openstudybuildertests-9497bfb181cb.json


In [88]:
bearer_token = !gcloud auth application-default print-access-token
bearer_token[0]

'ya29.c.b0AT7lpjBX-m8-DmM8Z73nL587oggOCrA3Qy4OXwjGbFzuVJbipaQPJdHcnBRRGj1U_s1fRtx3JwhG_Jkty6R2lJBTPRap_K8sPmYVM96O5ai6Uu1nA-w7qjkHyMyUln1YJ0O4Hzot-daPRz5sdXn9q3cSNxCW24G58Q8zWp1b9Q9rh6QNbh3gfSQJGuxE9B6M766nu0OxoxyuaStcIuklnelAC3bIr8c...............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

# Load Data from CT.gov

In [44]:
studies = pd.read_csv("diabetes_type_1_all_ct_studies.csv")
studies.columns

Index(['Rank', 'NCT Number', 'Title', 'Acronym', 'Status', 'Study Results',
       'Conditions', 'Interventions', 'Outcome Measures',
       'Sponsor/Collaborators', 'Gender', 'Age', 'Phases', 'Enrollment',
       'Funded Bys', 'Study Type', 'Study Designs', 'Other IDs', 'Start Date',
       'Primary Completion Date', 'Completion Date', 'First Posted',
       'Results First Posted', 'Last Update Posted', 'Locations',
       'Study Documents', 'URL'],
      dtype='object')

# Call GCP Healthcare NLP API on each Study title

In [93]:
gcp_hc_nlp_url = "https://healthcare.googleapis.com/v1/projects/openstudybuildertests/locations/europe-west4/services/nlp:analyzeEntities"
headers = {
    "Authorization": "Bearer " + bearer_token[0],
    "Content-Type": "application/json",
}
data = {
    "nlpService": "projects/openstudybuildertests/locations/europe-west4/services/nlp",
    "documentContent": "Evaluation of the biomedical and psychosocial impact of automated Closed-Loop (Artificial Pancreas) insulin delivery in women with type 1 diabetes during pregnancy"
}

In [94]:
studies["nlp"] = ""

In [95]:

for i, study in studies.iterrows():
    if study["nlp"] == "":
        title = study["Title"]
        id = study["NCT Number"]
        data["documentContent"] = title
        response = requests.post(gcp_hc_nlp_url, headers=headers, json=data)
        if response.status_code == 200:
            studies.at[i, "nlp"] = response.json()
        else:
            time.sleep(60)

KeyboardInterrupt: 

In [96]:
studies.to_csv("studies_with_concepts.csv", index=False)

In [98]:
studies

,Rank,NCT Number,Title,Acronym,Status,Study Results,Conditions,Interventions,Outcome Measures,Sponsor/Collaborators,...,Start Date,Primary Completion Date,Completion Date,First Posted,Results First Posted,Last Update Posted,Locations,Study Documents,URL,nlp
0,1,NCT04291703,Low Dose Antithymocyte Globulin (ATG) to Delay...,TN28,Not yet recruiting,No Results Available,"Diabetes Mellitus, Type 1",Drug: Antithymocyte Globulin|Drug: Placebo (fo...,Progression to Stage 3 T1D,National Institute of Diabetes and Digestive a...,...,December 2022,"December 31, 2028","December 31, 2029","March 2, 2020",NaN,"November 28, 2022",NaN,NaN,https://ClinicalTrials.gov/show/NCT04291703,"{'entityMentions': [{'mentionId': '1', 'type':..."
1,2,NCT05593081,Multicenter Study of Fulminant Type 1 Diabetes...,NaN,Recruiting,No Results Available,"Diabetes Mellitus, Type 1",Genetic: HLA,Change in serum hemoglobin A1c level|Change in...,Second Xiangya Hospital of Central South Unive...,...,"May 20, 2022","December 31, 2027","December 31, 2027","October 25, 2022",NaN,"October 25, 2022","Quanzhou First People's Hospital, Quanzhou, Fu...",NaN,https://ClinicalTrials.gov/show/NCT05593081,"{'entityMentions': [{'mentionId': '1', 'type':..."
2,3,NCT04616391,AHCL System Initiation in T1D Patients naïve t...,NaN,Unknown status,No Results Available,"Diabetes Mellitus, Type 1",Device: insulin pump Medtronic 780G,Between group TIR difference|Between group TIR...,Jagiellonian University|Medtronic Poland Spółk...,...,"November 2, 2020","March 31, 2021","July 1, 2021","November 5, 2020",NaN,"November 5, 2020",NaN,NaN,https://ClinicalTrials.gov/show/NCT04616391,{}
3,4,NCT04670198,A Psychosocial Education Programme for Young P...,YES,"Active, not recruiting",No Results Available,"Diabetes Mellitus, Type 1",Behavioral: Youth Empowerment Skills|Behaviora...,Glycated haemoglobin (HbA1c)|Frequency of bloo...,King's College London|Guy's and St Thomas' Fou...,...,"June 1, 2021","March 31, 2023","March 31, 2023","December 17, 2020",NaN,"April 7, 2022","Guy's and St Thomas' NHS Foundation Trust, Lon...",NaN,https://ClinicalTrials.gov/show/NCT04670198,"{'entityMentions': [{'mentionId': '1', 'type':..."
4,5,NCT02215200,ATG-GCSF in New Onset Type 1 Diabetes,ATG-GCSF,Completed,Has Results,"Diabetes Mellitus, Type 1",Drug: Anti-Thymocyte Globulin (ATG)|Drug: Gran...,Change in Area Under the Stimulated C-peptide ...,National Institute of Diabetes and Digestive a...,...,December 2014,August 2017,August 2018,"August 13, 2014","March 2, 2020","March 2, 2020","University of California - San Francisco, San ...","""Study Protocol"", https://ClinicalTrials.gov/P...",https://ClinicalTrials.gov/show/NCT02215200,"{'entityMentions': [{'mentionId': '1', 'type':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3132,3133,NCT00246844,Assessment of Insulin Production From Native P...,NaN,Completed,No Results Available,Long-Functioning Pancreas Allografts|Pancreas ...,NaN,NaN,National Institute of Diabetes and Digestive a...,...,"October 26, 2005",NaN,"August 15, 2007","October 30, 2005",NaN,"July 2, 2017","National Institutes of Health Clinical Center,...",NaN,https://ClinicalTrials.gov/show/NCT00246844,
3133,3134,NCT04419779,Evaluation of the Efficacy and Safety of Duode...,REVITALIZE 1,Recruiting,No Results Available,Type 2 Diabetes,Device: Duodenal Mucosal Resurfacing (DMR)|Dev...,The primary endpoint is the percentage of subj...,"Fractyl Health, Inc.|Fractyl Laboratories, Inc.",...,"March 8, 2021",June 2023,January 2025,"June 5, 2020",NaN,"October 17, 2022","Mayo Clinic Arizona, Scottsdale, Arizona, Unit...",NaN,https://ClinicalTrials.gov/show/NCT04419779,
3134,3135,NCT05132179,"Treating Diabetic Foot Ulcers, Comparing Two T...",NaN,Recruiting,No Results Available,Diabetic Foot Ulcer,"Drug: Dakins Full Strength Solution, 0.5% Topi...",Time to healing|Proportion of DFU that heals w...,Vastra Gotaland Region,...,"February 1, 2022","December 31, 2023","December 

In [92]:
response = requests.post(gcp_hc_nlp_url, headers=headers, json=data)
response.json()

{'error': {'code': 403,
  'message': 'SNOMEDCT_US vocabulary is not accessible from user location',
  'status': 'PERMISSION_DENIED'}}

In [99]:
print(len(studies[studies['nlp'] == '']))

1836
